## Ze względu na znaczące trudności w doprowadzeniu do końca oraz otrzymania jakkkolwiek wartościowych wyników w poprzedniej wersji projektu (rozpoznawanie mowy - temat numer 5 z PolEval 2019) postanowiłem zmienić temat na analizę prześladowań w internecie we wpisach na portalu tweeter (temat numer 6 PolEval 2019) ze względu na to, że już mam doświadczenie w podobnych zadaniach dzięki czemu jestem w stanie bardziej świadomie przeprowadzić proces oraz prawidłowo zinterpretować wyniki

#### W pierwszym etapie pobieram plik .zip udostępniony jako dane treningowe na stronie PolEval, rozpakowuję go i wczytuję zawartość (treść tweetów oraz etykiety) do zmiennych

In [1]:
import wget
import os
import zipfile
import string
import spacy
import numpy as np
from stop_words import get_stop_words
import re
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import (TextVectorization, Dense, Embedding, LSTM, 
                                     Conv1D,  MaxPooling1D, GlobalMaxPooling1D,
                                     Dropout, BatchNormalization, SpatialDropout1D,
                                    Bidirectional)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_auc_score
import keras
from tensorflow.keras.optimizers import Adam 


zip_file_name = 'task_6-1.zip'
tweets_contents_file = 'training_set_clean_only_text.txt'
tweets_labels_file = 'training_set_clean_only_tags.txt'
url = 'https://raw.githubusercontent.com/PatrycyD/INL_2/master/task_6-1.zip'
if not os.path.isfile(zip_file_name):
    wget.download(url) 

def extract_from_zip(zip_file, content_to_extract):
    with zipfile.ZipFile(zip_file) as z:
        with open(content_to_extract, 'wb') as f:
            f.write(z.read(content_to_extract))
            print('Extracted', content_to_extract)
            f.close()
        z.close()
        
def load_to_variable(file_to_load, data_type):
    if data_type == 'np.array':
        contents = np.array([]).reshape(1, -1)
    else:
        contents = []
    with open(file_to_load, 'r', encoding='utf-8') as file:
        for row in file:
            if data_type == 'np.array':
                contents = np.append(contents, row)
            else:
                contents.append(row)
    file.close()
    return contents


if not os.path.isfile(tweets_labels_file):
    extract_from_zip(zip_file_name, tweets_labels_file)

if not os.path.isfile(tweets_contents_file):
    extract_from_zip(zip_file_name, tweets_contents_file)
    
labels = load_to_variable(tweets_labels_file, 'np.array')
labels = labels.astype(np.float32)
tweets = load_to_variable(tweets_contents_file, 'list')
# tweets = tweets[:10]

#### Poniżej zdefiniowane są funkcje służące do przeprocesowania tekstu. Wyliczając po kolei od góry służą do:
- usunięcia słów 'stop words' - nie wnoszących wartości do tekstu
- usuwanie linków http
- usuwanie nazw użytkowników anonimowych
- usuwanie emotek
- usuwaniie znaków spoza tablicy ascii
- usuwanie znaków interpunkcyjnych
- usuwanie powtarzających się znaków
- usuwanie cyfr
- usuwanie nadmiarowych spacji
- lematyzacja

#### Następnie wszystkie te funkcje są wywoływane na wyżej wczytanych tekstach

In [2]:
stop_words = get_stop_words('polish')
def remove_stop_words(text):
    return ''.join([word for word in text if word not in stop_words])

def clean_URLs(text):
    return re.sub(r'http\S+', '', text)

def remove_nicknames(text):
    return text.replace('@anonymized_account', '').strip().strip('\n')

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def remove_emojis(text):
    return emoji_pattern.sub(r'', text)

def remove_non_ascii_chars(text):
    return re.sub(r'[^\x00-\x7f]',r'', text) 

punctuations_list = string.punctuation
def clean_punctuation(text):
    translator = str.maketrans('', '', punctuations_list) # jest to mapowanie i zamiana, dwa pierwsze argumenty to dwa stringi, gdzie znaki w pierwszym są zamieniane na znaki w drugim stringu, zgodnie z indeksem, Trzeci argument to znaki, które są mapowane do None => zostaną po prostu usunięte
    return text.translate(translator)

def clean_repeating_chars(text):
    return re.sub(r'(.)1+', r'1', text)

def clean_numbers(text):
    return re.sub('[0-9]+', '', text)

def remove_multiplied_spaces(text):
    return re.sub(' +', ' ', text)

# !pip install --upgrade spacy
# !python -m spacy download pl_core_news_sm
nlp = spacy.load('pl_core_news_sm')

def lemmatize(text):
    doc = nlp(text)
    lemmatized_sentence = ''
    for token in doc:
        lemmatized_sentence = f'{lemmatized_sentence} {token.lemma_}'
        
    return lemmatized_sentence.strip()

print('Removing stop words')
tweets = [remove_stop_words(tweet) for tweet in tweets]
print('Cleaning URLs')
tweets = [clean_URLs(tweet) for tweet in tweets]
print('Removing nicknames')
tweets = [remove_nicknames(tweet) for tweet in tweets]
print('Removing emojis')
tweets = [remove_emojis(tweet) for tweet in tweets]
print('Removing non ascii characters')
tweets = [remove_non_ascii_chars(tweet) for tweet in tweets]
print('Removing punctuation')
tweets = [clean_punctuation(tweet) for tweet in tweets] 
print('Removing repeating characters')
tweets = [clean_repeating_chars(tweet) for tweet in tweets]
print('Removing numbers')
tweets = [clean_numbers(tweet) for tweet in tweets]
print('Removing multiplied spaced')
tweets = [remove_multiplied_spaces(tweet) for tweet in tweets]
print('Lemmatizing')
tweets = [lemmatize(tweet) for tweet in tweets]

Removing stop words
Cleaning URLs
Removing nicknames
Removing emojis
Removing non ascii characters
Removing punctuation
Removing repeating characters
Removing numbers
Removing multiplied spaced
Lemmatizing


## Jeżeli tutaj pojawi się błąd spacy związany z pobieraniem paczki polskiej proszę o zresetowanie środowiska i uruchomienie wszystkiego od nowa - paczka może pobrać się prawidłowo po zaktualizowaniu spacy i zresetowaniu środowiska

#### Poniżej wczytuję przedtrenowany zbiór wektorów słów udostępniony przez Uniwersytet Łódzki. Zbiór ma ponad 930 000 wektorów

In [3]:
embedding_file = 'pl-embeddings-cbow.txt'

if not os.path.isfile(embedding_file):
    url = 'http://publications.ics.p.lodz.pl/2016/word_embeddings/pl-embeddings-cbow.txt'
    wget.download(url)
    
embeddings_index = {}
with open(embedding_file, encoding='utf-8') as file:
    for line in file:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs
        
embeddings_index.pop('933198') #na początku pliku jest notatka o liczbie wektorów i liczbie punktów w wektorach

print('Found %s word vectors.' % len(embeddings_index))

Found 933198 word vectors.


#### Poniżej pobieram dane testowe udostępnione na stronie PolEval i wczytuję je do odopwiednich zmiennych. Zrobię także preprocessing, tak żebym mógł użyć danych testowych do zrobienia słownika

In [4]:
def download_file_and_read_to_var(file_to_download, is_test):
    url = f'https://raw.githubusercontent.com/PatrycyD/INL_2/master/{file_to_download}'
    wget.download(url)
    with open(file_to_download, 'r', encoding='utf-8') as file:
        if is_test:
            contents = np.array([]).reshape(1, -1)
        else:
            contents = []
        for row in file:
            if is_test:
                contents = np.append(contents, int(row))
            else:
                contents.append(row)
                
        file.close()
        
    return contents
        
test_labels_file = 'test_set_clean_only_tags.txt'
test_text_file = 'test_set_clean_only_text.txt'

os.remove(test_labels_file)
y_test = download_file_and_read_to_var(test_labels_file, True)
    
os.remove(test_text_file)
X_test = download_file_and_read_to_var(test_text_file, False)

def call_all_preprocessing_functions(text_data):
    
    text_data = [remove_stop_words(tweet) for tweet in text_data]
    print('Cleaning URLs')
    text_data = [clean_URLs(tweet) for tweet in text_data]
    print('Removing nicknames')
    text_data = [remove_nicknames(tweet) for tweet in text_data]
    print('Removing emojis')
    text_data = [remove_emojis(tweet) for tweet in text_data]
    print('Removing non ascii characters')
    text_data = [remove_non_ascii_chars(tweet) for tweet in text_data]
    print('Removing punctuation')
    text_data = [clean_punctuation(tweet) for tweet in text_data] 
    print('Removing repeating characters')
    text_data = [clean_repeating_chars(tweet) for tweet in text_data]
    print('Removing numbers')
    text_data = [clean_numbers(tweet) for tweet in text_data]
    print('Removing multiplied spaced')
    text_data = [remove_multiplied_spaces(tweet) for tweet in text_data]
    print('Lemmatizing')
    text_data = [lemmatize(tweet) for tweet in text_data]
    
    return text_data

X_test = call_all_preprocessing_functions(X_test)

Cleaning URLs
Removing nicknames
Removing emojis
Removing non ascii characters
Removing punctuation
Removing repeating characters
Removing numbers
Removing multiplied spaced
Lemmatizing


#### Poniżej tworzę słownik wszystkich słów wraz z ich indeksami za pomocą kerasowego Vectorizera

In [5]:
max_tokens = 200000
vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=200)
vectorizer.adapt(tweets + X_test)

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

test = ['lipa', 'mecz', 'wyjazd', 'sezon']
[word_index[w] for w in test]

[16024, 40, 1153, 141]

#### Poniżej tworzę macierz wszystkich słów występujących w danych zastąpnionych wektorami pobranymi z przedtrenowanego modelu

In [6]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, idx in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[idx] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 10426 words (9110 misses)


#### Bardzo dużo słów nie znalazło odpowiedników w przedtrenowanym modelu. Najprawdopodoniej wynika to z faktu, iż występuje w tekście wiele wyrazów potocznych i mozliwe że specjalistycznych z zakresu piłki. Wiele osób może także popełniać błędy przy pisaniu i takich słów nie można przyporządkować. Może to obniżyć znacząco wyniki wytrenowanego modelu, jednak to najlepsze co udało mi się osiągnąć. Nie udało mi się także znaleźć większego polsiego przetrenowanego zbioru wektorów słów

#### Teraz przekronwertuję zbiór testowy do macierzy z wektorami słów

In [7]:
print('Vectorizing test data')
X_test = vectorizer(np.array([[s] for s in X_test])).numpy()

Vectorizing test data


#### Poniżej przekonwertowałem do macierzy słów zbiór testowy oraz wydzieliłem z niego niewielki zbiór walidacyjny. Następnie stworzyłem trzy modele: lasów losowych, maszycn wspierających wektory i k najbliższych sąsiadów, następnie wytrenowałem je na przygotowanym zbiorze

In [8]:
val_size = 0.15
X_train = vectorizer(np.array([[s] for s in tweets])).numpy()
X_train = X_train[:int(X_train.shape[0]*(1-val_size))]
X_val = X_train[int(X_train.shape[0]*(1-val_size)):]
y_train = np.array(labels)
y_train = y_train[:int(y_train.shape[0]*(1-val_size))]
y_val = y_train[int(y_train.shape[0]*(1-val_size)):]

rf = RandomForestClassifier(n_estimators=15, max_depth=5)
knn = KNeighborsClassifier(n_neighbors=20)
svc = SVC()

rf.fit(X_train, y_train)
knn.fit(X_train, y_train)
svc.fit(X_train, y_train)

rf_preds = rf.predict(X_test)
knn_preds = knn.predict(X_test)
svc_preds = svc.predict(X_test)

rf_train_preds = rf.predict(X_train)
knn_train_preds = knn.predict(X_train)
svc_train_preds = svc.predict(X_train)

In [9]:
print('Random Forest test predictions')
print(confusion_matrix(y_test, rf_preds))
print('\nK Nearest Neighbors test predictions')
print(confusion_matrix(y_test, knn_preds))
print('\nSupport Vector Machines test predictions')
print(confusion_matrix(y_test, svc_preds))

Random Forest test predictions
[[866   0]
 [134   0]]

K Nearest Neighbors test predictions
[[866   0]
 [134   0]]

Support Vector Machines test predictions
[[866   0]
 [134   0]]


In [10]:
print('Random Forest train predictions')
print(confusion_matrix(y_train, rf_train_preds))
print('\nK Nearest Neighbors train predictions')
print(confusion_matrix(y_train, knn_train_preds))
print('\nSupport Vector Machines train predictions')
print(confusion_matrix(y_train, svc_train_preds))

Random Forest train predictions
[[7923    0]
 [ 611    0]]

K Nearest Neighbors train predictions
[[7923    0]
 [ 611    0]]

Support Vector Machines train predictions
[[7923    0]
 [ 610    1]]


In [11]:
print(f'Ranom Forest test roc_auc: {round(roc_auc_score(y_test, rf_preds), 2)}')
print(f'Ranom Forest train roc_auc: {round(roc_auc_score(y_train, rf_train_preds), 2)}')

print(f'\nK Nearest Neighbors test roc_auc: {round(roc_auc_score(y_test, knn_preds), 2)}')
print(f'K Nearest Neighbors train roc_auc: {round(roc_auc_score(y_train, knn_train_preds), 2)}')

print(f'\nSupport Vector Machines test roc_auc: {round(roc_auc_score(y_test, svc_preds), 2)}')
print(f'Support Vector Machines train roc_auc: {round(roc_auc_score(y_train, svc_train_preds), 2)}')

Ranom Forest test roc_auc: 0.5
Ranom Forest train roc_auc: 0.5

K Nearest Neighbors test roc_auc: 0.5
K Nearest Neighbors train roc_auc: 0.5

Support Vector Machines test roc_auc: 0.5
Support Vector Machines train roc_auc: 0.5


### Wyniki ze wszystkich trzech modeli nie są jakkolwiek przydatne, wszystko zostało zaklasyfikowane jako klasa pozytywna

#### Stworzę teraz sieć neuronową w celu poprawy wyników. Zacznę od konwolucyjnej

In [12]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

cnn = keras.Sequential()
cnn.add(embedding_layer)
cnn.add(BatchNormalization(epsilon=1e-05, momentum=0.1))
cnn.add(Conv1D(128, 7, activation='relu', padding='same'))
cnn.add(MaxPooling1D(2))
cnn.add(Conv1D(128, 7, activation='relu', padding='same'))
cnn.add(GlobalMaxPooling1D())
cnn.add(Dropout(0.5))
cnn.add(Dense(100, activation='relu'))
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

# adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1953800   
                                                                 
 batch_normalization (BatchN  (None, None, 100)        400       
 ormalization)                                                   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         89728     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 128)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         114816    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0

In [13]:
cnn.fit(X_train, y_train, epochs=20, validation_data=[X_val, y_val], batch_size=128)

Epoch 1/20
67/67 [==============================] - 18s 255ms/step - loss: 0.2949 - auc: 0.5422 - val_loss: 0.3497 - val_auc: 0.6995
Epoch 2/20
67/67 [==============================] - 17s 251ms/step - loss: 0.2481 - auc: 0.6684 - val_loss: 0.2906 - val_auc: 0.8554
Epoch 3/20
67/67 [==============================] - 19s 278ms/step - loss: 0.2183 - auc: 0.7916 - val_loss: 0.2380 - val_auc: 0.9283
Epoch 4/20
67/67 [==============================] - 16s 232ms/step - loss: 0.1821 - auc: 0.8780 - val_loss: 0.1662 - val_auc: 0.9701
Epoch 5/20
67/67 [==============================] - 15s 226ms/step - loss: 0.1377 - auc: 0.9427 - val_loss: 0.1216 - val_auc: 0.9864
Epoch 6/20
67/67 [==============================] - 15s 227ms/step - loss: 0.1068 - auc: 0.9642 - val_loss: 0.0584 - val_auc: 0.9981
Epoch 7/20
67/67 [==============================] - 15s 230ms/step - loss: 0.0824 - auc: 0.9753 - val_loss: 0.0459 - val_auc: 0.9995
Epoch 8/20
67/67 [==============================] - 15s 223ms/step - 

In [14]:
#funkcja do wyliczenia klas z wyników prawdopodobieństw wychodzących z sieci
def return_classes_from_predictions(predictions):
    preds_classes = np.array([]).reshape(1,-1)
    for pred in predictions:
        if pred[0] >= 0.5:
            preds_classes = np.append(preds_classes, 1.)
        else:
            preds_classes = np.append(preds_classes, 0.)
            
    return preds_classes

In [15]:
cnn_preds = cnn.predict(X_test)
cnn_preds_classes = return_classes_from_predictions(cnn_preds)
cnn_train_preds = cnn.predict(X_train)
cnn_train_preds_classes = return_classes_from_predictions(cnn_train_preds)

print('CNN train predictions')
print(confusion_matrix(y_train, cnn_train_preds_classes))

print('\nCNN test predictions')
print(confusion_matrix(y_test, cnn_preds_classes))

print(f'\nCNN test roc_auc: {round(roc_auc_score(y_test, cnn_preds_classes), 2)}')
print(f'CNN test accuracy: {round(accuracy_score(y_test, cnn_preds_classes), 2)}')
print(f'\nCNN train roc_auc: {round(roc_auc_score(y_train, cnn_train_preds_classes), 2)}')
print(f'CNN train accuracy: {round(accuracy_score(y_train, cnn_train_preds_classes), 2)}')

CNN train predictions
[[7921    2]
 [   2  609]]

CNN test predictions
[[862   4]
 [125   9]]

CNN test roc_auc: 0.53

CNN test accuracy: 0.87
CNN train roc_auc: 1.0
CNN train accuracy: 1.0


#### Wyniki są zauważalnie lepsze. Wydaje się jednak że może być jakieś pole do poprawy. Spróbuję teraz zbudować sieć rekurencyjną i osiągnąć za jej pomocą lepsze wyniki

In [33]:
rnn = keras.Sequential()
rnn.add(embedding_layer)
# rnn.add(LSTM(100))
# # rnn.add(SpatialDropout1D(0.4))
# rnn.add(Dropout(0.6))
# rnn.add(Dense(128, activation='relu'))
# rnn.add(Dense(1, activation='sigmoid'))
rnn.add(Bidirectional(LSTM(64,  return_sequences=True)))
rnn.add(Bidirectional(LSTM(32)))
rnn.add(Dense(64, activation='relu'))
rnn.add(Dropout(0.5))
rnn.add(Dense(1, activation='sigmoid'))

rnn.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['AUC'])
rnn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1953800   
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 128)        84480     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 1)                

In [34]:
rnn.fit(X_train, y_train, epochs=5, validation_data=[X_val, y_val], batch_size=128)

Epoch 1/5
67/67 [==============================] - 58s 774ms/step - loss: 0.3056 - auc: 0.5148 - val_loss: 0.3547 - val_auc: 0.6706
Epoch 2/5
67/67 [==============================] - 53s 785ms/step - loss: 0.2381 - auc: 0.7039 - val_loss: 0.2779 - val_auc: 0.8763
Epoch 3/5
67/67 [==============================] - 52s 782ms/step - loss: 0.1795 - auc: 0.8778 - val_loss: 0.1865 - val_auc: 0.9452
Epoch 4/5
67/67 [==============================] - 52s 780ms/step - loss: 0.1343 - auc: 0.9357 - val_loss: 0.1227 - val_auc: 0.9758
Epoch 5/5
67/67 [==============================] - 58s 863ms/step - loss: 0.0888 - auc: 0.9725 - val_loss: 0.0738 - val_auc: 0.9901


In [ ]:
rnn_preds = rnn.predict(X_test)
rnn_preds_classes = return_classes_from_predictions(rnn_preds)
rnn_train_preds = rnn.predict(X_train)
rnn_train_preds_classes = return_classes_from_predictions(rnn_train_preds)

print('RNN train predictions')
print(confusion_matrix(y_train, rnn_train_preds_classes))

print('\nRNN test predictions')
print(confusion_matrix(y_test, rnn_preds_classes))

print(f'\nRNN test roc_auc: {round(roc_auc_score(y_test, rnn_preds_classes), 2)}')
print(f'RNN test accuracy: {round(accuracy_score(y_test, rnn_preds_classes), 2)}')
print(f'\nRNN train roc_auc: {round(roc_auc_score(y_train, rnn_train_preds_classes), 2)}')
print(f'RNN train accuracy: {round(accuracy_score(y_train, rnn_train_preds_classes), 2)}')

#### Wyniki z sieci rekurencyjnej są gorsze niż wyniki z sieci konwolucyjnej, dlatego należy uznać ją za najlepszy model. Możliwość poprawiania wyników może być ograniczona przez fakt dużej liczby słów które nie znalazły swoich odpowiedników w przedtrenowanym rozwiązaniu oraz ze względu na niewielki zbiór treningowy